# Data Cleaning

Now we have scraped our data from the Alcoholics Anonymous and Smoking subreddits, its time to clean out data!

### Import Relevant Libraries:

In [1]:
import pandas as pd
import re
import string
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 100) # More readable pandas output for huge column sizes

### Load our data files

In [2]:
data_alc = pd.read_csv('data/alcoholicsanonymous_raw.csv') # Alcoholic Anonymous Data
data_smoking = pd.read_csv('data/stopsmoking_raw.csv') # Stop Smoking Data

## Preliminary Check on data

Let's look at our data first:

In [3]:
data_alc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 71 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     2000 non-null   int64  
 1   all_awardings                  2000 non-null   object 
 2   allow_live_comments            2000 non-null   bool   
 3   author                         2000 non-null   object 
 4   author_flair_css_class         0 non-null      float64
 5   author_flair_richtext          1991 non-null   object 
 6   author_flair_text              0 non-null      float64
 7   author_flair_type              1991 non-null   object 
 8   author_fullname                1991 non-null   object 
 9   author_is_blocked              2000 non-null   bool   
 10  author_patreon_flair           1991 non-null   object 
 11  author_premium                 1991 non-null   object 
 12  awarders                       2000 non-null   o

In [4]:
data_smoking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 82 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     2000 non-null   int64  
 1   all_awardings                  2000 non-null   object 
 2   allow_live_comments            2000 non-null   bool   
 3   author                         2000 non-null   object 
 4   author_flair_css_class         197 non-null    object 
 5   author_flair_richtext          1993 non-null   object 
 6   author_flair_text              197 non-null    object 
 7   author_flair_type              1993 non-null   object 
 8   author_fullname                1993 non-null   object 
 9   author_is_blocked              2000 non-null   bool   
 10  author_patreon_flair           1993 non-null   object 
 11  author_premium                 1993 non-null   object 
 12  awarders                       2000 non-null   o

Seems we have a lot of columns, but we really only need the titles and the selftext. Also note that we have a media_only column, which could be used to remove data that only consists of medias and no text.

## Creating our Subreddit Column
---

There appears to be no column to indicate if the origin of the subreddit posts. Let's add one to set our classification variable for our machine learning model.

Alcoholic subreddit will be set as 1, and smoking is 0.

In [5]:
data_alc['subreddit'] = 1
data_alc['subreddit'] # Verify that all rows in subreddit column on alcoholic data is now set as 1

0       1
1       1
2       1
3       1
4       1
       ..
1995    1
1996    1
1997    1
1998    1
1999    1
Name: subreddit, Length: 2000, dtype: int64

In [6]:
data_smoking['subreddit'] = 0
data_smoking.info() # Verify that all rows in subreddit column on smoking data is now set as 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 82 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     2000 non-null   int64  
 1   all_awardings                  2000 non-null   object 
 2   allow_live_comments            2000 non-null   bool   
 3   author                         2000 non-null   object 
 4   author_flair_css_class         197 non-null    object 
 5   author_flair_richtext          1993 non-null   object 
 6   author_flair_text              197 non-null    object 
 7   author_flair_type              1993 non-null   object 
 8   author_fullname                1993 non-null   object 
 9   author_is_blocked              2000 non-null   bool   
 10  author_patreon_flair           1993 non-null   object 
 11  author_premium                 1993 non-null   object 
 12  awarders                       2000 non-null   o

## Merge both Dataframes together

Now that we have set our subreddit columns, let's merge both data together into a single dataframe by concatenating them.

Also note that the index will not be continuous once we have merged both dataframes together, so we will reset the index as well to maintain continuity in the index.

In [7]:
data_df = pd.concat([data_alc,data_smoking])
data_df.reset_index(inplace = True)
# We have to reset index to get a proper index for identifying the post

data_df
# We check the tail for 2 reasons, to verify the data is concatenated by seeing the total columns, and to check the last index.

,index,Unnamed: 0,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,...,thumbnail_width,author_flair_template_id,media,media_embed,secure_media,secure_media_embed,media_metadata,gallery_data,is_gallery,poll_data
0,0,0,[],False,dvlgxng616,NaN,[],NaN,text,t2_6ls6k8yz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,[],False,Emergency_Garden_149,NaN,[],NaN,text,t2_c19s1t82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,[],False,Yoself_reflection,NaN,[],NaN,text,t2_eczow0re,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,[],False,Essby1111,NaN,[],NaN,text,t2_bnv10h6a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,[],False,Okie69R,NaN,[],NaN,text,t2_5ltehq4u,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1995,1995,[],False,seathbydnudnu,NaN,[],NaN,text,t2_5mutdzid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3996,1996,1996,[],False,SenecaThePresent,NaN,[],NaN,text,t2_d1g6yhs0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3997,1997,1997,[],False,Dxrcyyy,NaN,[],NaN,text,t2_4k0eceyo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3998,1998,1998,[],False,2112CreateAcount,NaN,[],NaN,text,t2_biljxtvu,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Data Cleaning: Media Only Posts

Now that we have a full list, we can see we have a media_only column that may give us a good gage to whether the post is just a picture or media. We would want to remove this data as media only posts does not contain any text and is not useful for our machine learning model.

In [8]:
data_df_no_media = data_df[data_df.media_only == False] # Remove the row if media_only is True
data_df_no_media.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 83 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   index                          4000 non-null   int64  
 1   Unnamed: 0                     4000 non-null   int64  
 2   all_awardings                  4000 non-null   object 
 3   allow_live_comments            4000 non-null   bool   
 4   author                         4000 non-null   object 
 5   author_flair_css_class         197 non-null    object 
 6   author_flair_richtext          3984 non-null   object 
 7   author_flair_text              197 non-null    object 
 8   author_flair_type              3984 non-null   object 
 9   author_fullname                3984 non-null   object 
 10  author_is_blocked              4000 non-null   bool   
 11  author_patreon_flair           3984 non-null   object 
 12  author_premium                 3984 non-null   o

Appears that media data is not present in all our data points. However, this is a useful step in case any data appears in our future data if we were to model it against future posts.

## Remove Rows with Empty Selftexts
There are rows that have no selftexts at all. These rows should be removed as they will affect our model because our model takes in text inputs.

In [9]:
data_df_no_media_empty = data_df_no_media[data_df_no_media.selftext.notnull()] # Remove all empty selftext rows
data_df_no_media_empty.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3385 entries, 0 to 3998
Data columns (total 83 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   index                          3385 non-null   int64  
 1   Unnamed: 0                     3385 non-null   int64  
 2   all_awardings                  3385 non-null   object 
 3   allow_live_comments            3385 non-null   bool   
 4   author                         3385 non-null   object 
 5   author_flair_css_class         111 non-null    object 
 6   author_flair_richtext          3371 non-null   object 
 7   author_flair_text              111 non-null    object 
 8   author_flair_type              3371 non-null   object 
 9   author_fullname                3371 non-null   object 
 10  author_is_blocked              3385 non-null   bool   
 11  author_patreon_flair           3371 non-null   object 
 12  author_premium                 3371 non-null   o

## Filtering out only selftext and title columns
We are really only interested in the self text, titles and the subreddit column , so let's drop everything else by sending only these 2 columns into our new Dataframe.

In [10]:
self_title_df = pd.DataFrame()

self_title_df['selftext'] = data_df_no_media_empty['selftext']
self_title_df['title'] = data_df_no_media_empty['title']
self_title_df['subreddit'] = data_df_no_media_empty['subreddit']
# Add only the required columns into our new selt_title_df so that we don't have to drop every column
# This is to save processing power

self_title_df.head(20)

,selftext,title,subreddit
0,Been going pretty hard with drugs for a while recently quit and looking back at messages and pos...,Does anyone look back at messages and post's you have sent and cringe.,1
1,[removed],19 year old do I have a problem?,1
3,"So, does anyone know if you can go to rehab and still keep your job if you work from home?\n\nI ...",Rehab and working from home???,1
5,I didn’t know how to phrase the question so I will try to clarify: my[26F] brother [21m] has bee...,What would you have wanted your sibling to say when they noticed there was a problem with alcohol?,1
6,[removed],Advice on my alcohol consumption!,1
7,"I'm not trying to sell y'all insurance. Rather, I've been looking at hosting an event (meditatio...","What sort of ""liability insurance"" do y'all have at your events?",1
8,[removed],How do you deal with guilt?,1
9,Im 29m and coming up on 11 months sober now and im almost done with my 4th step. Ive done the st...,Does it actually get better?,1
10,I was sober for 3 years until this past June. I started drinking after life took a tumble. I dra...,Day two of sobriety,1
11,Hey everyone..I’m about alcoholic. I down a full bottle of wine daily for the past 4 years. Afte...,First day,1


Appears we have some posts that are removed or deleted, lets remove them as well.

In [11]:
df_clean = pd.DataFrame()

self_title_df = self_title_df[self_title_df.selftext != '[removed]'] # Remove all removed posts
df_clean = self_title_df[self_title_df.selftext != '[deleted]'] # remove all deleted posts
df_clean.reset_index(inplace = True) # Reset the index again to get our proper indexing for the cleaned data
df_clean.drop('index', axis = 1, inplace = True) # Drop the old index column
df_clean

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,selftext,title,subreddit
0,Been going pretty hard with drugs for a while recently quit and looking back at messages and pos...,Does anyone look back at messages and post's you have sent and cringe.,1
1,"So, does anyone know if you can go to rehab and still keep your job if you work from home?\n\nI ...",Rehab and working from home???,1
2,I didn’t know how to phrase the question so I will try to clarify: my[26F] brother [21m] has bee...,What would you have wanted your sibling to say when they noticed there was a problem with alcohol?,1
3,"I'm not trying to sell y'all insurance. Rather, I've been looking at hosting an event (meditatio...","What sort of ""liability insurance"" do y'all have at your events?",1
4,Im 29m and coming up on 11 months sober now and im almost done with my 4th step. Ive done the st...,Does it actually get better?,1
...,...,...,...
3293,I still get some cravings. But in each case I can find that my desire to smoke would appear to b...,I had my last cigarette on the 30th March 2021. Today is the 30th August. So I haven’t smoked a ...,0
3294,"Since March 2020, I have been off and on quitting. I went four months then went back when I star...",Emotions and Relationships when quitting.,0
3295,"I quit almost a year ago to the day, it was amazing how radically diffrent things became after....","My journey, and where I'm at now.",0
3296,"Hey all, I'll start by saying I'm 16 and from the UK, but as many of you know, getting fags is n...",Smoking really damaging me at a young age,0


## Saving our Data
Now our data is cleaned, let's save our data to csv.

In [12]:
df_clean.to_csv('data/cleaned_data.csv')

With our data cleaned, we can proceed to the next portion of our analysis, our Exploratory Data Analysis and Model. This will be covered on the next notebook.